In [1]:
import os 
import dotenv
dotenv.load_dotenv()
MLFLOW_TRACKING_URI = os.environ.get('MLFLOW_TRACKING_URI')
MLFLOW_TRACKING_USERNAME = os.environ.get('MLFLOW_TRACKING_USERNAME')
MLFLOW_TRACKING_PASSWORD = os.environ.get('MLFLOW_TRACKING_PASSWORD')





In [2]:
import os
%pwd

'/Users/wa_c/Desktop/Machine Learning/ Projects/End to end project (Udemy)/datascienceproject/research'

In [3]:
os.chdir("../")
%pwd

'/Users/wa_c/Desktop/Machine Learning/ Projects/End to end project (Udemy)/datascienceproject'

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [5]:
@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [6]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml,create_directories,save_json

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH
                 
                 ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:   
        config=self.config.model_evaluation  # Reading from config.yaml file
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir]) #Reading from config.yaml file

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri= "https://dagshub.com/syedwassiulhaque/datascienceproject.mlflow"

        )
        return model_evaluation_config

In [8]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib


In [9]:
""" 


class ModelEvaluation:
    def __init__(self, config:ModelEvaluationConfig):
        self.config = config

    def eval_metrics (self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual,pred)
        return rmse, mae, r2  

    ## Logging data to mlflow
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme  

        with mlflow.start_run():
            
            predicted_qualities = model.predict(test_x)
            (rmse, mae, r2) = self. eval_metrics(test_y, predicted_qualities)

            #Saving metrics as local

            scores = {"rmse": rmse, "mae": mae,"r2": r2} 
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)
            
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            #Model registry does not work with file store 
            if tracking_url_type_store != "file":

                #Register the model
                #There are other ways to use the Model Registry, which depends on the use case,
                # Documentation for more info: 
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNetModel")

            else:
                mlflow.sklearn.log_model(model, "model")










"""

' \n\n\nclass ModelEvaluation:\n    def __init__(self, config:ModelEvaluationConfig):\n        self.config = config\n\n    def eval_metrics (self, actual, pred):\n        rmse = np.sqrt(mean_squared_error(actual, pred))\n        mae = mean_absolute_error(actual, pred)\n        r2 = r2_score(actual,pred)\n        return rmse, mae, r2  \n\n    ## Logging data to mlflow\n    def log_into_mlflow(self):\n\n        test_data = pd.read_csv(self.config.test_data_path)\n        model = joblib.load(self.config.model_path)\n\n        test_x = test_data.drop([self.config.target_column], axis=1)\n        test_y = test_data[[self.config.target_column]]\n\n        mlflow.set_registry_uri(self.config.mlflow_uri)\n        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme  \n\n        with mlflow.start_run():\n            \n            predicted_qualities = model.predict(test_x)\n            (rmse, mae, r2) = self. eval_metrics(test_y, predicted_qualities)\n\n            #Saving metri

In [10]:
 
class ModelEvaluation:
    def __init__(self, config:ModelEvaluationConfig):
        self.config = config

    def eval_metrics (self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual,pred)
        return rmse, mae, r2     



    def log_into_mlflow(self):
            # Ensure the directory for the metrics file exists
            Path(self.config.metric_file_name).parent.mkdir(parents=True, exist_ok=True)

            # Rest of your code...
            test_data = pd.read_csv(self.config.test_data_path)
            model = joblib.load(self.config.model_path)

            test_x = test_data.drop([self.config.target_column], axis=1)
            test_y = test_data[[self.config.target_column]]

            mlflow.set_registry_uri(self.config.mlflow_uri)
            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme  

            with mlflow.start_run():
                predicted_qualities = model.predict(test_x)
                (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

                # Save metrics as local
                scores = {"rmse": rmse, "mae": mae, "r2": r2}
                save_json(path=Path(self.config.metric_file_name), data=scores)

                # Log metrics into MLflow
                mlflow.log_params(self.config.all_params)
                mlflow.log_metric("rmse", rmse)
                mlflow.log_metric("r2", r2)
                mlflow.log_metric("mae", mae)

                # Model registry
                if tracking_url_type_store != "file":
                    mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNetModel")
                else:
                    mlflow.sklearn.log_model(model, "model")


In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()

except Exception as e:
    raise e    

[2024-10-21 14:35:00,594: INFO: common: yaml file: config/config.yaml loaded succesfully]
[2024-10-21 14:35:00,595: INFO: common: yaml file: params.yaml loaded succesfully]
[2024-10-21 14:35:00,597: INFO: common: yaml file: schema.yaml loaded succesfully]
[2024-10-21 14:35:00,597: INFO: common: created directory at: artifacts]
[2024-10-21 14:35:00,598: INFO: common: created directory at: articats/model_evaluation]
[2024-10-21 14:35:01,166: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]


2024/10/21 14:35:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'ElasticNetModel' already exists. Creating a new version of this model...
2024/10/21 14:35:07 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticNetModel, version 3
Created version '3' of model 'ElasticNetModel'.
2024/10/21 14:35:08 INFO mlflow.tracking._tracking_service.client: 🏃 View run youthful-vole-749 at: https://dagshub.com/syedwassiulhaque/datascienceproject.mlflow/#/experiments/0/runs/ce73d4f4ed544060a85b40e7f5b265ee.
2024/10/21 14:35:08 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/syedwassiulhaque/datascienceproject.mlflow/#/experiments/0.
